In [2]:
library(nichenetr)
library(Seurat)
library(SeuratObject)
library(tidyverse)
library(dplyr)
library(circlize)
library(networkD3)

In [3]:
lr_network = readRDS("../../nichenet_mouse_pre_compiled_interactions_networks/lr_network_mouse_21122021.rds")
ligand_target_matrix = readRDS("../../nichenet_mouse_pre_compiled_interactions_networks/ligand_target_matrix_nsga2r_final_mouse.rds")
weighted_networks = readRDS("../../nichenet_mouse_pre_compiled_interactions_networks/weighted_networks_nsga2r_final_mouse.rds")
ligand_tf_matrix = readRDS("../../nichenet_mouse_pre_compiled_interactions_networks/ligand_tf_matrix_nsga2r_final_mouse.rds")
sig_network = readRDS("../../nichenet_mouse_pre_compiled_interactions_networks/signaling_network_mouse_21122021.rds")
gr_network = readRDS("../../nichenet_mouse_pre_compiled_interactions_networks/gr_network_mouse_21122021.rds")

weighted_networks_lr = weighted_networks$lr_sig %>% inner_join(lr_network, by = c("from","to"))
weighted_networks_lr=weighted_networks_lr[,-c(4,5)]

In [4]:
pvat_24weeks_control_male_expr = read.csv("../exported_data/pvat_24weeks_control_male_expr.csv", row.names=1)
pvat_24weeks_control_male_metadata = read.csv("../exported_data/pvat_24weeks_control_male_metadata.csv", row.names=1)

In [5]:
head(pvat_24weeks_control_male_expr)

,X1700066B19Rik,X3110082J24Rik,X4930404H24Rik,X4930455H04Rik,X4933400A11Rik,X6330411D24Rik,A1cf,A2m,A2ml1,A3galt2,⋯,Zyg11a,Zyg11b,Zyx,Zzef1,Zzz3,hist1h2ail2,mrpl11,mrpl24,mrpl9,rnf141
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
AAACCCAAGGTTACCT-1-6,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,1,0,0,0,0,0,1
AAACCCACAGGCCCTA-1-6,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,1,2,0,0,0,0,1
AAACCCACATGAGTAA-1-6,0,0,0,0,0,0,0,0,0,0,⋯,0,0,1,0,0,0,0,0,0,0
AAACGAACATAATCGC-1-6,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
AAACGCTGTATGGTTC-1-6,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
AAACGCTTCATGCCCT-1-6,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,1,0,0,0,0,0,0


In [6]:
head(pvat_24weeks_control_male_metadata)

,sample_id,n_genes_by_counts,log1p_n_genes_by_counts,total_counts,log1p_total_counts,pct_counts_in_top_20_genes,total_counts_mt,log1p_total_counts_mt,pct_counts_mt,total_counts_ribo,⋯,systolic_bp_7W,MAP_9W,systolic_bp_9W,MAP_16W,systolic_bp_16W,MAP_24W,systolic_bp_24W,doublet,celltype,celltype_broad
,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>
AAACCCAAGGTTACCT-1-6,P_CTRL_24W_M_01,978,6.886532,1395,7.241366,10.46595,6,1.945910,0.4301075,2,⋯,NA,108.5,148.5,115.7,154,110.6,163,False,ECs_Lymph,Endothelial_Cells
AAACCCACAGGCCCTA-1-6,P_CTRL_24W_M_01,2561,7.848543,6279,8.745125,9.60344,7,2.079442,0.1114827,10,⋯,NA,108.5,148.5,115.7,154,110.6,163,False,Adipocytes_Brown,Adipocytes
AAACCCACATGAGTAA-1-6,P_CTRL_24W_M_01,1543,7.342132,2954,7.991254,13.84563,12,2.564949,0.4062288,2,⋯,NA,108.5,148.5,115.7,154,110.6,163,False,Pericytes,Pericytes
AAACGAACATAATCGC-1-6,P_CTRL_24W_M_01,922,6.827629,1347,7.206377,12.10097,4,1.609438,0.2969562,6,⋯,NA,108.5,148.5,115.7,154,110.6,163,False,Fibroblasts_Bmper+_Nrxn1+,Fibroblasts
AAACGCTGTATGGTTC-1-6,P_CTRL_24W_M_01,1046,6.953684,1610,7.384610,10.74534,7,2.079442,0.4347826,3,⋯,NA,108.5,148.5,115.7,154,110.6,163,False,ECs_Cap,Endothelial_Cells
AAACGCTTCATGCCCT-1-6,P_CTRL_24W_M_01,2055,7.628518,4352,8.378621,12.59191,12,2.564949,0.2757353,8,⋯,NA,108.5,148.5,115.7,154,110.6,163,False,Adipocytes_Brown,Adipocytes


In [7]:
seuratObj = CreateSeuratObject(counts = t(pvat_24weeks_control_male_expr), meta.data = pvat_24weeks_control_male_metadata)  # counts should be a cell by gene matrix and not gene by cell

Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Data is of class matrix. Coercing to dgCMatrix.”


In [8]:
table(seuratObj@meta.data$celltype_broad)


       Adipocytes Endothelial_Cells       Fibroblasts      Immune_Cells 
             3547               588               788               413 
Mesothelial_Cells    Neuronal_Cells         Pericytes              SMCs 
              212                 4               288                 3 

In [8]:
cell_data <- list(seuratObj = seuratObj,
                  expression_matrix = t(pvat_24weeks_control_male_expr),
                  transposed_expression_matrix = pvat_24weeks_control_male_expr
                 )

In [9]:
# List nicheNet analysis components using mouse preknowledge cell-cell communication data.
nichenet_networks <- list(
  lr_network = lr_network,
  ligand_target_matrix = ligand_target_matrix,
  weighted_networks = weighted_networks,
  ligand_tf_matrix = ligand_tf_matrix,
  sig_network = sig_network,
  gr_network = gr_network,
  weighted_networks_lr = weighted_networks_lr
)

In [10]:
# Identify genes with high expression levels across cells.
highly_expressed_genes <- function(expression_matrix, cell_ids) {
    
    transposed_expression_matrix = t(expression_matrix)
    
    genes <- transposed_expression_matrix[cell_ids, ] %>%
    apply(2, function(x) {10 * (2^x - 1)}) %>%
    apply(2, function(x) {log2(mean(x) + 1)}) %>%
    .[. >= 2] %>%
    names()
  return(genes)
    
    }

In [11]:
# Generate a heatmap for active ligand-target interactions using specified ligands and targets.
ligand_target_heatmap <- function(active_ligand_target_links_df, ligand_target_matrix, best_upstream_ligands) {

    active_ligand_target_links = prepare_ligand_target_visualization(ligand_target_df = active_ligand_target_links_df, ligand_target_matrix = ligand_target_matrix, cutoff = 0.33)
    order_ligands = intersect(best_upstream_ligands, colnames(active_ligand_target_links)) %>% rev() %>% make.names()
    order_targets = active_ligand_target_links_df$target %>% unique() %>% intersect(rownames(active_ligand_target_links)) %>% make.names()
    rownames(active_ligand_target_links) = rownames(active_ligand_target_links) %>% make.names() # make.names() for heatmap visualization of genes like H2-T23
    colnames(active_ligand_target_links) = colnames(active_ligand_target_links) %>% make.names() # make.names() for heatmap visualization of genes like H2-T23
    vis_ligand_target = active_ligand_target_links[order_targets,order_ligands] %>% t()
    options(repr.plot.width=20, repr.plot.height=10)
    #p_ligand_target_network = vis_ligand_target %>% make_heatmap_ggplot("Prioritized ligands","Predicted target genes", color = "purple",legend_position = "top", x_axis_position = "top",legend_title = "Regulatory potential")  + theme(axis.text.x = element_text(face = "italic")) + scale_fill_gradient2(low = "whitesmoke",  high = "purple", breaks = c(0,0.0045,0.0090))
    #p_ligand_target_network = p_ligand_target_network + theme(axis.text.x = element_text(face = "italic", size = 20),
    #                                    axis.text.y = element_text(size = 20),
    #                                   axis.title.x = element_text(size = 20),
    #                                   axis.title.y = element_text(size = 20),
    #                                   plot.title = element_text(hjust = 0.5))

    return(vis_ligand_target)
    
}

In [12]:
# Construct and order a heatmap for top ligand-receptor interactions
ligand_receptor_heatmap <- function(lr_network, best_upstream_ligands, expressed_receptors, weighted_networks_lr) {

    lr_network_top = lr_network %>% filter(from %in% best_upstream_ligands & to %in% expressed_receptors) %>% distinct(from,to)
    best_upstream_receptors = lr_network_top %>% pull(to) %>% unique()
    lr_network_top_df_large = weighted_networks_lr %>% filter(from %in% best_upstream_ligands & to %in% best_upstream_receptors)
    lr_network_top_df = lr_network_top_df_large %>% spread("from","weight",fill = 0)
    lr_network_top_matrix = lr_network_top_df %>% select(-to) %>% as.matrix() %>% magrittr::set_rownames(lr_network_top_df$to)
    dist_receptors = dist(lr_network_top_matrix, method = "binary")
    hclust_receptors = hclust(dist_receptors, method = "ward.D2")
    order_receptors = hclust_receptors$labels[hclust_receptors$order]
    dist_ligands = dist(lr_network_top_matrix %>% t(), method = "binary")
    hclust_ligands = hclust(dist_ligands, method = "ward.D2")
    order_ligands_receptor = hclust_ligands$labels[hclust_ligands$order]
    order_receptors = order_receptors %>% intersect(rownames(lr_network_top_matrix))
    order_ligands_receptor = order_ligands_receptor %>% intersect(colnames(lr_network_top_matrix))
    vis_ligand_receptor_network = lr_network_top_matrix[order_receptors, order_ligands_receptor]
    rownames(vis_ligand_receptor_network) = order_receptors %>% make.names()
    colnames(vis_ligand_receptor_network) = order_ligands_receptor %>% make.names()
    options(repr.plot.width=20, repr.plot.height=10)
    #p_ligand_receptor_network = vis_ligand_receptor_network %>% t() %>% make_heatmap_ggplot("Ligands","Receptors", color = "mediumvioletred", x_axis_position = "top",legend_title = "Prior interaction potential")
    #p_ligand_receptor_network = p_ligand_receptor_network + theme(axis.text.x = element_text(face = "italic", size = 20), 
    #                                    axis.text.y = element_text(size = 20),
    #                                   axis.title.x = element_text(size = 20),
    #                                   axis.title.y = element_text(size = 20)) 
    return(vis_ligand_receptor_network)
    }

In [13]:
# Analyze ligand-receptor activities between specified source and target cell types using nicheNet networks and cell data.

ligand_receptor_activites <- function(source_celltype, target_celltype, cell_data, nichenet_networks) {
    
    lr_network = nichenet_networks$lr_network
    ligand_target_matrix = nichenet_networks$ligand_target_matrix
    weighted_networks = nichenet_networks$weighted_networks
    ligand_tf_matrix = nichenet_networks$ligand_tf_matrix
    sig_network = nichenet_networks$sig_network
    gr_network = nichenet_networks$gr_network
    weighted_networks_lr = nichenet_networks$weighted_networks_lr
    seuratObj = cell_data$seuratObj
    expression_matrix = cell_data$expression_matrix
    transposed_expression_matrix = cell_data$transposed_expression_matrix

    n = 500
    #number_of_target_genes = n
    
    source_cell_ids = WhichCells(seuratObj, expression = celltype_broad == source_celltype)
    target_cell_ids = WhichCells(seuratObj, expression = celltype_broad == target_celltype)
    
    source_genes = highly_expressed_genes(expression_matrix, source_cell_ids)
    target_genes = highly_expressed_genes(expression_matrix, target_cell_ids)
    #n = length(target_genes)-1
    
    Idents(seuratObj) = seuratObj$celltype
    
    background_expressed_genes = target_genes %>% .[. %in% rownames(ligand_target_matrix)]
    ligands = lr_network %>% pull(from) %>% unique()
    expressed_ligands = intersect(ligands, source_genes)
    receptors = lr_network %>% pull(to) %>% unique()
    expressed_receptors = intersect(receptors, target_genes)
    potential_ligands = lr_network %>% filter(from %in% expressed_ligands & to %in% expressed_receptors) %>% pull(from) %>% unique()
    
    ligand_activities = predict_ligand_activities(geneset = (target_genes[0:n]), background_expressed_genes = background_expressed_genes, ligand_target_matrix = ligand_target_matrix, potential_ligands = potential_ligands)
    ligand_activities = ligand_activities %>% arrange(-aupr_corrected)
    best_upstream_ligands = ligand_activities %>% arrange(-aupr_corrected) %>% pull(test_ligand)
    best_upstream_ligands = best_upstream_ligands %>% intersect(expressed_ligands)

    
    ligand_expression_tbl = tibble(
    ligand = best_upstream_ligands, 
    source = transposed_expression_matrix[source_cell_ids, best_upstream_ligands] %>% apply(2,function(x){10*(2**x - 1)}) %>% apply(2,function(x){log2(mean(x) + 1)}))

    general_ligands = ligand_expression_tbl %>% pull(ligand)
    
    ligand_type_indication_df = tibble(
                                ligand_type = c(rep(source_celltype, times = general_ligands %>% length())),
                                ligand = c(general_ligands))


    active_ligand_target_links_df = best_upstream_ligands %>% lapply(get_weighted_ligand_target_links,geneset = target_genes[0:n], ligand_target_matrix = ligand_target_matrix, n = 250) %>% bind_rows()
    active_ligand_target_links_df = active_ligand_target_links_df %>% mutate(target_type = target_celltype) %>% inner_join(ligand_type_indication_df) 
    active_ligand_target_links_df = na.omit(active_ligand_target_links_df)
    

    p_ligand_target_network = ligand_target_heatmap(active_ligand_target_links_df, ligand_target_matrix, best_upstream_ligands)
    p_ligand_receptor_network = ligand_receptor_heatmap(lr_network, best_upstream_ligands, expressed_receptors, weighted_networks_lr)




    return(list(ligand_activities = ligand_activities, 
            p_ligand_target_network = p_ligand_target_network, 
            p_ligand_receptor_network = p_ligand_receptor_network))


}

In [14]:
table(seuratObj@meta.data$celltype_broad)


       Adipocytes Endothelial_Cells       Fibroblasts      Immune_Cells 
             7500              2422               660               324 
Mesothelial_Cells    Neuronal_Cells         Pericytes              SMCs 
              125                19               376                31 

In [15]:
cell_types <- c(
  "Adipocytes",       
  "Endothelial_Cells",     
  "Fibroblasts",            
  "Immune_Cells",  
  "Mesothelial_Cells",  
  "Neuronal_Cells",      
  "Pericytes",   
  "SMCs" 
)

num <- length(cell_types)
print(paste("Number of cell types:", num))

[1] "Number of cell types: 8"


In [16]:
pb <- txtProgressBar(min = 0,      # Minimum value of the progress bar
                     max = length(cell_types)^2, # Assuming you iterate through each pair
                     style = 3,    # Progress bar style (also available style = 1 and style = 2)
                     width = 50,   # Progress bar width. Defaults to getOption("width")
                     char = "=")   # Character used to create the bar

i = 1
pvat_24weeks_control_male_nichenet = list()

for (source_cell in cell_types) {
  for (target_cell in cell_types) {

      key = paste(gsub(" ", "", source_cell), "2", gsub(" ", "", target_cell), sep="")
      
      pvat_24weeks_control_male_nichenet[[key]] <- tryCatch({
        ligand_receptor_activites(source_cell, target_cell, cell_data, nichenet_networks)
      }, error = function(e) {
        cat("Error with", source_cell, "to", target_cell, ":", e$message, "\n")
        NULL 
      })
    
    i = i + 1
    setTxtProgressBar(pb, i)
  }
}
close(pb)

  |                                                  |   0%

Joining with `by = join_by(ligand)`


  |==                                                |   3%

Joining with `by = join_by(ligand)`


  |==                                                |   5%

...

Joining with `by = join_by(ligand)`


  |================================================= |  98%

Joining with `by = join_by(ligand)`


  |==================================================| 100%

Joining with `by = join_by(ligand)`


In [19]:
names_interactions <- names(pvat_24weeks_control_male_nichenet)
for (interaction_name in names_interactions) {
  interaction <- pvat_24weeks_control_male_nichenet[[interaction_name]]
  
  matrix_names <- c("ligand_activities", "p_ligand_receptor_network", "p_ligand_target_network")
  for (matrix_name in matrix_names) {
    tryCatch({
      if (!is.null(interaction[[matrix_name]])) {
        interaction[[matrix_name]] <- as.data.frame(interaction[[matrix_name]])
      }
    }, error = function(e) {
       cat("Issue converting", matrix_name, "in", interaction_name, "to a data.frame:", e$message, "\n")
    })
  }
  pvat_24weeks_control_male_nichenet[[interaction_name]] <- interaction
}

In [20]:
saveRDS(pvat_24weeks_control_male_nichenet ,'pvat_24weeks_control_male_nichenet.rds')